# IMPORTACIÓN DE LIBRERIAS

In [75]:
import pandas as pd
import time
import smtplib
from email.mime.application import MIMEApplication
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import math  # Necesario para verificar NaN
import os
import pywhatkit
import time
import pyautogui

# EXTRACCIÓN O LLAMADO DE MI DATA FRAME

In [76]:
# Leer el registro desde un archivo Excel
data_df = pd.read_excel(r'C:\Users\jalopezz\Desktop\Pruebas_Excels\Envio_Backlog.xlsx', dtype={"Numero": str, "loyaltyPhone": str,"loyaltyPhone": str, "EnlaceAAA":str})

# Convertir el archivo Excel en un DataFrame de Pandas
info_backlog = pd.DataFrame(data_df)


# SELECCIONANDO COLUMNAS DE MI DATA FRAME

In [77]:
# Seleccionar columnas por nombre
info_backlog_ = info_backlog[['deliveryOrderNumber', 'trackingCode','lastStatusGSC','seller_ID','seller_Name', 'sku', 'description', 'segmento', 'carrierCode','punto_dropoff','direccion_dropoff','nombre_dropoff','horario_dropoff','frecuencia_dropoff','promisedBySeller', 'accion', 'Correo', 'Numero', 'loyaltyPhone', 'Speech','Estatusenvio','EnlaceAAA']]
print(info_backlog['Correo'])

0     rserna@falabella.com
1     rserna@falabella.com
2     rserna@falabella.com
3     rserna@falabella.com
4     rserna@falabella.com
5     rserna@falabella.com
6     rserna@falabella.com
7     rserna@falabella.com
8     rserna@falabella.com
9     rserna@falabella.com
10    rserna@falabella.com
11    rserna@falabella.com
12    rserna@falabella.com
13    rserna@falabella.com
14    rserna@falabella.com
Name: Correo, dtype: object


# AGREGANDO LOS FILTROS Y/O CONDICIONES ENVIO CORREO

In [78]:
# Correo sellers AAA que no son dropoff
info_backlog_1 = info_backlog[(info_backlog['segmento'] == 'AAA') & 
                              (~info_backlog['accion'].isin(['No accionar', 'Cancelar', 'Cambio de estado','Comunicar envío día siguiente']))&
                              (info_backlog['carrierCode']!= 'dropoff')]


# Correo Other Sellers Hoy estas en Ruta, Retraso & Pending
info_backlog_2 = info_backlog[(info_backlog['segmento'] != 'AAA') & 
                              (info_backlog['Speech'] == 'Speech_2') & 
                              (~info_backlog['accion'].isin(['No accionar', 'Cancelar', 'Cambio de estado','Comunicar envío día siguiente'])) &
                              (info_backlog['carrierCode']!= 'dropoff')]


# Correo Other Sellers Hoy estas en Ruta y Retraso
info_backlog_3 = info_backlog[(info_backlog['segmento'] != 'AAA') & 
                              (info_backlog['Speech'] == 'Speech_3') & 
                              (~info_backlog['accion'].isin(['No accionar', 'Cancelar', 'Cambio de estado','Comunicar envío día siguiente']))&
                              (info_backlog['carrierCode']!= 'dropoff')]


# Correo Other Sellers only Pending
info_backlog_4 = info_backlog[(info_backlog['segmento'] != 'AAA') & 
                              (info_backlog['Speech'] == 'Speech_4') & 
                              (~info_backlog['accion'].isin(['No accionar', 'Cancelar', 'Cambio de estado','Comunicar envío día siguiente']))]


#Correo DROP OFF - OTHERS SELLERS
info_backlog_6 = info_backlog[(info_backlog['segmento'] != 'AAA') & (info_backlog['carrierCode']== 'dropoff') &
                              info_backlog['Estatusenvio'].isin(['A tiempo','1 dia de retraso', '2 dias de retraso', '3 dias de retraso']) & 
                              (~info_backlog['accion'].isin(['No accionar', 'Cancelar', 'Cambio de estado','Comunicar envío día siguiente']))]
#Correo DROP OFF - AAA
info_backlog_7 = info_backlog[(info_backlog['segmento'] == 'AAA') & (info_backlog['carrierCode']== 'dropoff') &(info_backlog['lastStatusGSC'] != 'pending') &
                              info_backlog['Estatusenvio'].isin(['A tiempo','1 dia de retraso', '2 dias de retraso', '3 dias de retraso']) & 
                              (~info_backlog['accion'].isin(['No accionar', 'Cancelar', 'Cambio de estado','Comunicar envío día siguiente']))]


#Whats App sellers AAA con retraso
info_backlog_5 = info_backlog[(info_backlog['segmento'] == 'AAA') & 
                              info_backlog['Estatusenvio'].isin(['1 dia de retraso', '2 dias de retraso', '3 dias de retraso']) & 
                              (~info_backlog['accion'].isin(['No accionar', 'Cancelar', 'Cambio de estado','Comunicar envío día siguiente'])) &
                              (info_backlog['carrierCode'] != 'dropoff')]



# BLOQUE PARA ENVIAR MAILS A AAA

In [29]:
#BLOQUE PARA ENVIAR MAILS A AAA
import time
##import pyautogui
import smtplib
from email.mime.application import MIMEApplication
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import math  # Necesario para verificar NaN
import os


correo_usuario = 'aoriveram@falabella.com'
contrasena = 'Marinette2023.'
asunto_base = 'F.COM | 🚨 HOY ESTÁS EN RUTA🚨'

# Establecer la conexión con el servidor SMTP de Outlook
server = smtplib.SMTP('smtp.office365.com', 587)
server.starttls()

# Iniciar sesión en el servidor SMTP
server.login(correo_usuario, contrasena)

# Iterar sobre las tiendas únicas en el DataFrame
for seller in info_backlog_1['seller_ID'].unique():
    # Filtrar el DataFrame para la tienda actual
    df_seller = info_backlog_1[info_backlog_1['seller_ID'] == seller]


    # Obtener la dirección de correo electrónico de la tienda actual
    correo_seller = df_seller['Correo'].iloc[0]
    # Obtener el número de teléfono asignado al vendedor actual
    phone_asignado = df_seller['loyaltyPhone'].iloc[0]
    link_phone = df_seller['EnlaceAAA'].iloc[0]

    # Verificar si la dirección de correo electrónico es NaN
    if pd.isna(correo_seller):
        print(f"La dirección de correo electrónico para la tienda {seller} es nula (NaN). No se enviará el correo.")
        continue  # Saltar a la siguiente iteración

    # Obtener el valor de seller_Name para la tienda actual
    seller_name = df_seller['seller_Name'].iloc[0]

    # Seleccionar columnas específicas para mostrar en el cuerpo del correo
    columnas_seleccionadas = ['seller_Name', 'trackingCode','deliveryOrderNumber', 'sku', 'description', 'promisedBySeller', 'Estatusenvio']
    df_seller_seleccionada = df_seller[columnas_seleccionadas].drop_duplicates()
    df_seller_seleccionada  =  df_seller_seleccionada .sort_values(by='Estatusenvio', ascending=False)
    
    # Renombrar las columnas
    df_seller_seleccionada = df_seller_seleccionada.rename(columns={
    'seller_Name': 'Nombre del seller',
    'deliveryOrderNumber': 'Número de orden',
    'trackingCode': 'Rastreo',
    'description': 'Descripción',
    'promisedBySeller': 'Fecha de promesa',
    'Estatusenvio': 'Estado del envío'
    })
    
    # Configurar el mensaje
    msg = MIMEMultipart()
    msg['From'] = 'fcomlogisticasx@falabella.com'
    msg['To'] = correo_seller
    msg['Subject'] = asunto_base.format(seller)
    msg['CC'] = 'soporteperufcom@falabella.com'

    # Crear el cuerpo del correo en formato HTML
    mensaje_html = f"""
<html>
  <head>
    <style>
      table {{
        border-collapse: collapse;
        width: 100%;
      }}
      th, td {{
        border: 1px solid black;
        padding: 8px;
        text-align: left;
      }}
      th {{
        background-color: orange;
        color: white;
        font-size: 16px;
        font-weight: bold;
        text-align: center;
      }}
    </style>
  </head>
  <body>
    <p><h2><strong>Hola {seller_name}, con ID {seller}!</strong> Estas son las órdenes que recolectaremos hoy: </h2></p>
    
    <br>
    {df_seller_seleccionada.to_html(index=False)}
    <br>
    <p>Podrás revisar tus órdenes en falabella seller center desde <a href= "https://sellercenter.falabella.com/order">aquí</a>.</p>
    <p>Para gestionar tus pedidos a tiempo es importante:</p>
        <ol>
            <li>Imprimir y pegar la <em>etiqueta</em> en el exterior de tu empaque. Se visualiza al lado izquierdo del número de orden como <em>Label</em>.</li>
            <li>Hacer clic en <em>Listo para despachar</em> - como máximo un día antes de la fecha límite (antes de las 10:00PM).</li>
        </ol>
        
    <p>🚚 Revisa nuestro tutorial de despacho: <a href="https://youtu.be/lpk1_y9uYDg">Tutorial de despacho</a>.</p>
        
    <p>Siguiendo estos pasos estaremos programando la recolección de tus órdenes en la fecha límite de despacho. Recuerda que el tiempo de espera del operador logístico es de 10 minutos. </p>

    <p>🌟 Si necesitas soporte personalizado a través de WhatsApp, contáctanos en el {phone_asignado} o <a href="{link_phone}">ingresando al siguiente link</a>.</p>
    
    <p>¡Gracias!</p> 
  </body>
</html>
"""

    # Adjuntar el cuerpo del correo en formato HTML
    msg.attach(MIMEText(mensaje_html, 'html'))

    # Enviar el correo electrónico
    server.sendmail(correo_usuario, correo_seller, msg.as_string())

# Cerrar la conexión con el servidor SMTP
server.quit()



(221, b'2.0.0 Service closing transmission channel')

# BLOQUE PARA ENVIAR MAILS A OTHERS SELLERS (MAÑANA ESTAS EN RUTA Y RETRASO) & PENDING

In [30]:

# Convertir todas las columnas a cadenas para evitar el error de 'float' object has no attribute 'encode'
info_backlog_2= info_backlog_2.applymap(lambda x: str(x))

correo_usuario = 'aoriveram@falabella.com'
contrasena = 'Marinette2023.'
asunto_base = 'F.COM | 🚨 HOY ESTÁS EN RUTA🚨'

# Establecer la conexión con el servidor SMTP de Outlook
server = smtplib.SMTP('smtp.office365.com', 587)
server.starttls()

# Iniciar sesión en el servidor SMTP
server.login(correo_usuario, contrasena)


# Iterar sobre las tiendas únicas en el DataFrame
for seller in info_backlog_2['seller_ID'].unique():
    # Filtrar el DataFrame para la tienda actual
    df_seller_1 = info_backlog_2[info_backlog_2['seller_ID'] == seller]

    # Obtener la dirección de correo electrónico de la tienda actual
    correo_seller = df_seller_1['Correo'].iloc[0]

    # Verificar si la dirección de correo electrónico es NaN
    if isinstance(correo_seller, float) and math.isnan(correo_seller):
        print(f"La dirección de correo electrónico para la tienda {seller} es nula (NaN). No se enviará el correo.")
        continue  # Saltar a la siguiente iteración
   # Obtener el valor de seller_ID para la tienda actual
    seller_name = df_seller_1['seller_Name'].iloc[0]

    # Seleccionar columnas específicas para mostrar en el cuerpo del correo
    columnas_seleccionadas_2 = ['seller_Name', 'deliveryOrderNumber', 'trackingCode','sku', 'description', 'promisedBySeller', 'Estatusenvio']
    df_seller_seleccionada_2 = df_seller_1[columnas_seleccionadas_2].drop_duplicates()
    
    # Ordenar el DataFrame por la columna 'accion' de manera descendente
    df_seller_seleccionada_2 = df_seller_seleccionada_2.sort_values(by='Estatusenvio', ascending=False)

    # Renombrar las columnas
    df_seller_seleccionada_2 = df_seller_seleccionada_2.rename(columns={
    'seller_Name': 'Nombre del seller',
    'deliveryOrderNumber': 'Número de orden',
    'trackingCode': 'Rastreo',
    'description': 'Descripción',
    'promisedBySeller': 'Fecha de promesa',
    'Estatusenvio': 'Estado del envío'
    })
    # Configurar el mensaje
    msg = MIMEMultipart()
    msg['From'] = 'fcomlogisticasx@falabella.com'
    msg['To'] = correo_seller
    msg['Subject'] = asunto_base.format(seller)
    msg['CC'] = 'soporteperufcom@falabella.com'

    # Crear el cuerpo del correo en formato HTML
    mensaje_html = f"""
<html>
  <head>
    <style>
      table {{
        border-collapse: collapse;
        width: 100%;
      }}
      th, td {{
        border: 1px solid black;
        padding: 8px;
        text-align: left;
      }}
      th {{
        background-color: orange;
        color: white;
        font-size: 16px;
        font-weight: bold;
        text-align: center;
      }}
    </style>
  </head>
   <body>
    <p><h2><strong>Hola {seller_name}, con ID {seller}!</strong> Estas son las órdenes que recolectaremos hoy  </h2></p></p>
    
    <br>
    {df_seller_seleccionada_2.to_html(index=False)}
    <br>
    <p>*Tener en cuenta que si tienes ordenes con días de retraso ya están impactando el cumplimiento de tus indicadores. Recuerda que tu nivel de cumplimiento se relaciona directamente con la cantidad cobrada por cofinanciamiento logístico. Si necesitas más detalles sobre estos cobros, haz clic aqui :<a href="https://www.falabellasellers.com/comisiones-y-costos">https://www.falabellasellers.com/comisiones-y-costos</a></li></p>
    <p>*Las ordenes con 3 días de retraso de no ser entregadas será cancelada el día de hoy.</p>
    <br>

    <p><h2>🚨 🚨 Por otro lado, detectamos que tienes <strong>órdenes pendientes</strong> por colocar "listo para despachar"</h2></p>
    <p>Recuerda que debes cambiar el estado de estas órdenes a Listo para despachar un día previo de la fecha límite de despacho, como máximo hasta las 10 pm, para que podamos incluirlas en la ruta de recolección</p>
    
    <p>Queremos reforzar contigo el proceso para gestionarlas:</p>
    <ol>
        <li>Podrás revisar tus órdenes en falabella seller center desde <a href= "https://sellercenter.falabella.com/order">aquí</a>.</li>
        <li>Imprimir y pegar la <em>etiqueta</em> en el exterior de tu empaque. Se visualiza al lado izquierdo del número de orden como <em>Label</em>.</li>
        <li>Hacer clic en <em>Listo para despachar</em> - como máximo un día antes de la fecha límite (antes de las 10:00PM).</li>
    </ol>
    <p>🚚 Revisa nuestro tutorial de despacho: <a href="https://youtu.be/lpk1_y9uYDg">Tutorial de despacho</a>.</p>
    
    <p>¡No pierdas ventas! Recuerda que si la orden se cancela no habrá lugar a reclamo.</p>

    <p>Siguiendo estos pasos estaremos programando la recolección de estos pedidos para el día siguiente. Recuerda que, al llegar a tu almacén, el tiempo de espera del operador logístico es de 10 minutos.</p>
    
    <p><strong>¡Importante!</strong> </p>
        <li>Las órdenes con retraso impactarán en tu indicador de envíos a tiempo afectando tu puntaje operacional. Considera que el cofinanciamiento logístico varía de acuerdo a tu desempeño. Para más información sobre haz clic <a href= "https://www.falabellasellers.com/comisiones-y-costos">aquí</a>.</li>
        <li>Las ordenes con 3 días de retraso de no ser entregadas el día de hoy,serán canceladas.</li>
        <li>Este correo es un refuerzo. Siempre debes entregar los pedidos gestionados a tiempo en su respectiva fecha límite de despacho.</li>
    
    <p>¡Contamos con tu apoyo para mejorar la experiencia de nuestros clientes!</p>
        
  </body>
</html>
"""

      # Adjuntar el cuerpo del correo en formato HTML
    msg.attach(MIMEText(mensaje_html, 'html'))

    # Enviar el correo electrónico
    server.sendmail(correo_usuario, msg['To'], msg.as_string())

# Cerrar la conexión con el servidor SMTP
server.quit()


(221, b'2.0.0 Service closing transmission channel')

# BLOQUE PARA ENVIAR MAILS A OTHERS SELLERS (MAÑANA ESTAS EN RUTA Y RETRASO)

In [48]:
import smtplib
import math
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Convertir todas las columnas a cadenas para evitar el error de 'float' object has no attribute 'encode'
info_backlog_3 = info_backlog_3.applymap(lambda x: str(x))

correo_usuario = 'aoriveram@falabella.com'
contrasena = 'Marinette2023.'
asunto_base = 'F.COM | 🚨 HOY ESTÁS EN RUTA🚨'

# Función para enviar correos electrónicos
def enviar_correo(correo_usuario, contrasena, correo_destinatario, mensaje):
    try:
        # Establecer la conexión con el servidor SMTP de Outlook
        server = smtplib.SMTP('smtp.office365.com', 587, timeout=30)
        server.starttls()
        
        # Iniciar sesión en el servidor SMTP
        server.login(correo_usuario, contrasena)
        
        # Enviar el correo electrónico
        server.sendmail(correo_usuario, correo_destinatario, mensaje.as_string())
        
        print("Correo electrónico enviado exitosamente.")
    except Exception as e:
        print(f"Error al enviar el correo electrónico: {e}")
    finally:
        # Cerrar la conexión con el servidor SMTP
        server.quit()

# Iterar sobre las tiendas únicas en el DataFrame
for seller in info_backlog_3['seller_ID'].unique():
    # Filtrar el DataFrame para la tienda actual
    df_seller_2 = info_backlog_3[info_backlog_3['seller_ID'] == seller]

    # Obtener la dirección de correo electrónico de la tienda actual
    correo_seller = df_seller_2['Correo'].iloc[0]

    # Verificar si la dirección de correo electrónico es NaN
    if isinstance(correo_seller, float) and math.isnan(correo_seller):
        print(f"La dirección de correo electrónico para la tienda {seller} es nula (NaN). No se enviará el correo.")
        continue  # Saltar a la siguiente iteración
    
    # Obtener el valor de seller_ID para la tienda actual
    seller_name = df_seller_2['seller_Name'].iloc[0]

    # Seleccionar columnas específicas para mostrar en el cuerpo del correo
    columnas_seleccionadas_3 = ['seller_Name', 'deliveryOrderNumber', 'trackingCode','sku', 'description', 'promisedBySeller', 'Estatusenvio']
    df_seller_seleccionada_3 = df_seller_2[columnas_seleccionadas_3].drop_duplicates()
    
    # Ordenar el DataFrame por la columna 'accion' de manera descendente
    df_seller_seleccionada_3 = df_seller_seleccionada_3.sort_values(by='Estatusenvio', ascending=False)
    
    # Renombrar las columnas
    df_seller_seleccionada_3 = df_seller_seleccionada_3.rename(columns={
    'seller_Name': 'Nombre del seller',
    'deliveryOrderNumber': 'Número de orden',
    'trackingCode': 'Rastreo',
    'description': 'Descripción',
    'promisedBySeller': 'Fecha de promesa',
    'Estatusenvio': 'Estado del envío'
    })

    # Configurar el mensaje
    msg = MIMEMultipart()
    msg['From'] = 'fcomlogisticasx@falabella.com'
    msg['To'] = correo_seller
    msg['Subject'] = asunto_base.format(seller)
    msg['CC'] = 'soporteperufcom@falabella.com'

    # Crear el cuerpo del correo en formato HTML
    mensaje_html = f"""
<html>
  <head>
    <style>
      table {{
        border-collapse: collapse;
        width: 100%;
      }}
      th, td {{
        border: 1px solid black;
        padding: 8px;
        text-align: left;
      }}
      th {{
        background-color: orange;
        color: white;
        font-size: 16px;
        font-weight: bold;
        text-align: center;
      }}
    </style>
  </head>
   <body>
    <p><h2><strong>Hola {seller_name}, con ID {seller}!</strong> Estas son las órdenes que recolectaremos hoy</h2></p></p>
    
    <br>
    {df_seller_seleccionada_3.to_html(index=False)}
    <br>
    <p>*Tener en cuenta que si tienes ordenes con días de retraso ya están impactando el cumplimiento de tus indicadores. Recuerda que tu nivel de cumplimiento se relaciona directamente con la cantidad cobrada por cofinanciamiento logístico. Si necesitas más detalles sobre estos cobros, haz clic aqui :<a href="https://www.falabellasellers.com/comisiones-y-costos">https://www.falabellasellers.com/comisiones-y-costos</a></li></p>
    <p>*Las ordenes con 3 días de retraso de no ser entregadas será cancelada el día de hoy.</p>

        <p>*Recuerde: que no es necesario el envío de este correo para la recolección de estas órdenes. Asimismo, la unidad tiene un tiempo de espera de atención de 10 minutos. De marcarse el fallo y ser recurrente, se procede a evaluar ciertas penalidades sin opción a reclamo.</p>
     
    <p><strong>¡Importante!</strong> </p>
        <li>El operador logístico te llamará al llegar a tu almacén. El tiempo de espera es de 10 minutos, recuerda tener listos tu productos y cargo de entrega.</li>
        <li>Las órdenes con retraso impactarán en tu indicador de envíos a tiempo afectando tu puntaje operacional. Considera que el cofinanciamiento logístico varía de acuerdo a tu desempeño. Para más información sobre haz clic <a href= "https://www.falabellasellers.com/comisiones-y-costos">aquí</a>.</li>
        <li>Las ordenes con 3 días de retraso de no ser entregadas el día de hoy,serán canceladas.</li>
        <li>Este correo es un refuerzo. Siempre debes entregar los pedidos gestionados a tiempo en su respectiva fecha límite de despacho.</li>
    
    <p>¡Contamos con tu apoyo para mejorar la experiencia de nuestros clientes!</p>
        
  </body>
</html>
"""

    # Adjuntar el cuerpo del correo en formato HTML
    msg.attach(MIMEText(mensaje_html, 'html'))

    # Enviar el correo electrónico llamando a la función definida anteriormente
    enviar_correo(correo_usuario, contrasena, correo_seller, msg)


SMTPServerDisconnected: please run connect() first

# BLOQUE PARA ENVIAR MAILS ONLY PENDINGS

In [32]:
#BLOQUE PARA ENVIAR MAILS ONLY PENDINGS

# Convertir todas las columnas a cadenas para evitar el error de 'float' object has no attribute 'encode'
info_backlog_4= info_backlog_4.applymap(lambda x: str(x))

correo_usuario = 'aoriveram@falabella.com'
contrasena = 'Marinette2023.'
asunto_base = 'F.COM | 🚨 GESTIONA TUS ÓRDENES PENDIENTES 🚨'

# Establecer la conexión con el servidor SMTP de Outlook
server = smtplib.SMTP('smtp.office365.com', 587)
server.starttls()

# Iniciar sesión en el servidor SMTP
server.login(correo_usuario, contrasena)


# Iterar sobre las tiendas únicas en el DataFrame
for seller in info_backlog_4['seller_ID'].unique():
    # Filtrar el DataFrame para la tienda actual
    df_seller_3 = info_backlog_4[info_backlog_4['seller_ID'] == seller]

    # Obtener la dirección de correo electrónico de la tienda actual
    correo_seller = df_seller_3['Correo'].iloc[0]

    # Verificar si la dirección de correo electrónico es NaN
    if isinstance(correo_seller, float) and math.isnan(correo_seller):
        print(f"La dirección de correo electrónico para la tienda {seller} es nula (NaN). No se enviará el correo.")
        continue  # Saltar a la siguiente iteración
   # Obtener el valor de seller_ID para la tienda actual
    seller_name = df_seller_3['seller_Name'].iloc[0]

    # Seleccionar columnas específicas para mostrar en el cuerpo del correo
    columnas_seleccionadas_4 = ['seller_Name', 'deliveryOrderNumber','trackingCode','sku', 'description', 'promisedBySeller', 'Estatusenvio']
    df_seller_seleccionada_4 = df_seller_3[columnas_seleccionadas_4].drop_duplicates()
    
    # Ordenar el DataFrame por la columna 'accion' de manera descendente
    df_seller_seleccionada_4 = df_seller_seleccionada_4.sort_values(by='Estatusenvio', ascending=False)
    
    # Renombrar las columnas
    df_seller_seleccionada_4 = df_seller_seleccionada_4.rename(columns={
    'seller_Name': 'Nombre del seller',
    'deliveryOrderNumber': 'Número de orden',
    'trackingCode': 'Rastreo',
    'description': 'Descripción',
    'promisedBySeller': 'Fecha de promesa',
    'Estatusenvio': 'Estado del envío'
    })

    # Configurar el mensaje
    msg = MIMEMultipart()
    msg['From'] = 'fcomlogisticasx@falabella.com'
    msg['To'] = correo_seller
    msg['Subject'] = asunto_base.format(seller)
    msg['CC'] = 'soporteperufcom@falabella.com'

    # Crear el cuerpo del correo en formato HTML
    mensaje_html = f"""
<html>
  <head>
    <style>
      table {{
        border-collapse: collapse;
        width: 100%;
      }}
      th, td {{
        border: 1px solid black;
        padding: 8px;
        text-align: left;
      }}
      th {{
        background-color: orange;
        color: white;
        font-size: 16px;
        font-weight: bold;
        text-align: center;
      }}
    </style>
  </head>
   <body>
    <p><h2><strong>Hola {seller_name}, con ID {seller}! NO has colocado el “listo para despachar” </strong> en las siguientes órdenes:</h2></p></p>
    
    <br>
    {df_seller_seleccionada_4.to_html(index=False)}
    <br>

    <p>Queremos reforzar contigo el proceso para gestionarlas:</p>
    <ol>
        <li>Podrás revisar tus órdenes en falabella seller center desde <a href= "https://sellercenter.falabella.com/order">aquí</a>.</li>
        <li>Imprimir y pegar la <em>etiqueta</em> en el exterior de tu empaque. Se visualiza al lado izquierdo del número de orden como <em>Label</em>.</li>
        <li>Hacer clic en <em>Listo para despachar</em> - como máximo un día antes de la fecha límite (antes de las 10:00PM).</li>
    </ol>
    
    <p>🚚 Revisa nuestro tutorial de despacho: <a href="https://youtu.be/lpk1_y9uYDg">Tutorial de despacho</a>.</p>
    
    
    <p><strong>¡Importante!</strong> </p>
    <p>Es importante entregar todos los pedidos en su respectiva fecha límite de despacho, evita el retraso en tus entregas.</p>
        <li>Las órdenes con retraso impactarán en tu indicador de envíos a tiempo afectando tu puntaje operacional. Considera que el cofinanciamiento logístico varía de acuerdo a tu desempeño. Para más información sobre haz clic <a href= "https://www.falabellasellers.com/comisiones-y-costos">aquí</a>.</li>
        <li>Las ordenes con 3 días de retraso de no ser entregadas el día de hoy,serán canceladas.</li>
        
    <p>Este correo es un refuerzo. Siempre debes entregar los pedidos gestionados a tiempo en su respectiva fecha límite de despacho.</p>
    
    <p>¡Contamos con tu apoyo para mejorar la experiencia de nuestros clientes!</p>
        
  </body>
</html>
"""

      # Adjuntar el cuerpo del correo en formato HTML
    msg.attach(MIMEText(mensaje_html, 'html'))

    # Enviar el correo electrónico
    server.sendmail(correo_usuario, msg['To'], msg.as_string())

# Cerrar la conexión con el servidor SMTP
server.quit()


(221, b'2.0.0 Service closing transmission channel')

## BLOQUE PARA ENVIAR CORREO SELLERS DROP OFF A OTHERS

In [33]:


# Convertir todas las columnas a cadenas para evitar el error de 'float' object has no attribute 'encode'
info_backlog_6= info_backlog_6.applymap(lambda x: str(x))

correo_usuario = 'aoriveram@falabella.com'
contrasena = 'Marinette2023.'
asunto_base = 'F.COM | 🚨 ENTREGA TUS PEDIDOS HOY 🚨'

# Establecer la conexión con el servidor SMTP de Outlook
server = smtplib.SMTP('smtp.office365.com', 587)
server.starttls()

# Iniciar sesión en el servidor SMTP
server.login(correo_usuario, contrasena)


# Iterar sobre las tiendas únicas en el DataFrame
for seller in info_backlog_6['seller_ID'].unique():
    # Filtrar el DataFrame para la tienda actual
    df_seller_5 = info_backlog_6[info_backlog_6['seller_ID'] == seller]

    # Obtener la dirección de correo electrónico de la tienda actual
    correo_seller = df_seller_5['Correo'].iloc[0]

    # Verificar si la dirección de correo electrónico es NaN
    if isinstance(correo_seller, float) and math.isnan(correo_seller):
        print(f"La dirección de correo electrónico para la tienda {seller} es nula (NaN). No se enviará el correo.")
        continue  # Saltar a la siguiente iteración
   # Obtener el valor de seller_ID para la tienda actual
    seller_name = df_seller_5['seller_Name'].iloc[0]
    punto_dropoff = df_seller_5['punto_dropoff'].iloc[0]
    direccion =df_seller_5['direccion_dropoff'].iloc[0]
    tienda = df_seller_5['nombre_dropoff'].iloc[0]
    horario = df_seller_5['horario_dropoff'].iloc[0]
    frecuencia = df_seller_5['frecuencia_dropoff'].iloc[0]
    
    

    # Seleccionar columnas específicas para mostrar en el cuerpo del correo
    columnas_seleccionadas_5 = ['seller_Name', 'deliveryOrderNumber','trackingCode','sku', 'description', 'promisedBySeller', 'Estatusenvio']
    df_seller_seleccionada_5 = df_seller_5[columnas_seleccionadas_5].drop_duplicates()
    
    # Ordenar el DataFrame por la columna 'accion' de manera descendente
    df_seller_seleccionada_5 = df_seller_seleccionada_5.sort_values(by='Estatusenvio', ascending=False)
    
    # Renombrar las columnas
    df_seller_seleccionada_5 = df_seller_seleccionada_5.rename(columns={
    'seller_Name': 'Nombre del seller',
    'deliveryOrderNumber': 'Número de orden',
    'trackingCode': 'Rastreo',
    'description': 'Descripción',
    'promisedBySeller': 'Fecha de promesa',
    'Estatusenvio': 'Estado del envío'
    })

    # Configurar el mensaje
    msg = MIMEMultipart()
    msg['From'] = 'fcomlogisticasx@falabella.com'
    msg['To'] = correo_seller
    msg['Subject'] = asunto_base.format(seller)
    msg['CC'] = 'soporteperufcom@falabella.com'

    # Crear el cuerpo del correo en formato HTML
    mensaje_html = f"""
<html>
  <head>
    <style>
      table {{
        border-collapse: collapse;
        width: 100%;
      }}
      th, td {{
        border: 1px solid black;
        padding: 8px;
        text-align: left;
      }}
      th {{
        background-color: orange;
        color: white;
        font-size: 16px;
        font-weight: bold;
        text-align: center;
      }}
    </style>
  </head>
  <body>
    <p><h2><strong>Hola {seller_name}, con ID {seller}! Estas son las órdenes que debes entregar hoy en tu punto de acopio asignado:</strong></h2></p></p>
    
    <br>
    {df_seller_seleccionada_5.to_html(index=False)}
    <br>

    <strong><p>Datos sobre tu punto de acopio:</strong></p>
    <ol>
        <li>Nombre del punto: {punto_dropoff}</li>
        <li>Tienda: {tienda}</li>
        <li>-	Dirección: {direccion}</li>
        <li>Horario de entrega: {horario}</li>
        <li>Frecuencia: {frecuencia}</li>
    </ol>
    
    <p>🚚 Revisa nuestro tutorial de despacho: <a href="https://youtu.be/lpk1_y9uYDg">Tutorial de despacho</a>.</p>
    
    
    <p><strong>¡Importante!</strong> </p>
        <li>Recuerda que el punto de acopio debe firmar el cargo de entrega que le proporciones.</li>
        <li>Las ordenes con 3 días de retraso de no ser entregadas el día de hoy,serán canceladas.</li>
        <li>Este correo es un refuerzo. Siempre debes entregar los pedidos gestionados a tiempo en su respectiva fecha límite de despacho.</li>
        
    
    <p>¡Contamos con tu apoyo para mejorar la experiencia de nuestros clientes!</p>
        
  </body>
</html>
  """

      # Adjuntar el cuerpo del correo en formato HTML
    msg.attach(MIMEText(mensaje_html, 'html'))

    # Enviar el correo electrónico
    server.sendmail(correo_usuario, msg['To'], msg.as_string())

# Cerrar la conexión con el servidor SMTP
server.quit()

(221, b'2.0.0 Service closing transmission channel')

## BLOQUE PARA ENVIO CORREO AAA DROP OFF

In [34]:
# Convertir todas las columnas a cadenas para evitar el error de 'float' object has no attribute 'encode'
info_backlog_7= info_backlog_7.applymap(lambda x: str(x))

correo_usuario = 'aoriveram@falabella.com'
contrasena = 'Marinette2023.'
asunto_base = 'F.COM | 🚨 ENTREGA TUS PEDIDOS HOY 🚨'

# Establecer la conexión con el servidor SMTP de Outlook
server = smtplib.SMTP('smtp.office365.com', 587)
server.starttls()

# Iniciar sesión en el servidor SMTP
server.login(correo_usuario, contrasena)


# Iterar sobre las tiendas únicas en el DataFrame
for seller in info_backlog_7['seller_ID'].unique():
    # Filtrar el DataFrame para la tienda actual
    df_seller_6 = info_backlog_7[info_backlog_7['seller_ID'] == seller]

    # Obtener la dirección de correo electrónico de la tienda actual
    correo_seller = df_seller_6['Correo'].iloc[0]
    phone_asignado = df_seller_6['loyaltyPhone'].iloc[0]
    link_phone = df_seller_6['EnlaceAAA'].iloc[0]

    # Verificar si la dirección de correo electrónico es NaN
    if isinstance(correo_seller, float) and math.isnan(correo_seller):
        print(f"La dirección de correo electrónico para la tienda {seller} es nula (NaN). No se enviará el correo.")
        continue  # Saltar a la siguiente iteración
   # Obtener el valor de seller_ID para la tienda actual
    seller_name = df_seller_6['seller_Name'].iloc[0]
    punto_dropoff = df_seller_6['punto_dropoff'].iloc[0]
    direccion =df_seller_6['direccion_dropoff'].iloc[0]
    tienda = df_seller_6['nombre_dropoff'].iloc[0]
    horario = df_seller_6['horario_dropoff'].iloc[0]
    frecuencia = df_seller_6['frecuencia_dropoff'].iloc[0]
    
    

    # Seleccionar columnas específicas para mostrar en el cuerpo del correo
    columnas_seleccionadas_6 = ['seller_Name', 'deliveryOrderNumber','trackingCode','sku', 'description', 'promisedBySeller', 'Estatusenvio']
    df_seller_seleccionada_6 = df_seller_6[columnas_seleccionadas_6].drop_duplicates()
    
    # Ordenar el DataFrame por la columna 'accion' de manera descendente
    df_seller_seleccionada_6 = df_seller_seleccionada_6.sort_values(by='Estatusenvio', ascending=False)
    
    # Renombrar las columnas
    df_seller_seleccionada_6 = df_seller_seleccionada_6.rename(columns={
    'seller_Name': 'Nombre del seller',
    'deliveryOrderNumber': 'Número de orden',
    'trackingCode': 'Rastreo',
    'description': 'Descripción',
    'promisedBySeller': 'Fecha de promesa',
    'Estatusenvio': 'Estado del envío'
    })

    # Configurar el mensaje
    msg = MIMEMultipart()
    msg['From'] = 'fcomlogisticasx@falabella.com'
    msg['To'] = correo_seller
    msg['Subject'] = asunto_base.format(seller)
    msg['CC'] = 'soporteperufcom@falabella.com'

    # Crear el cuerpo del correo en formato HTML
    mensaje_html = f"""
<html>
  <head>
    <style>
      table {{
        border-collapse: collapse;
        width: 100%;
      }}
      th, td {{
        border: 1px solid black;
        padding: 8px;
        text-align: left;
      }}
      th {{
        background-color: orange;
        color: white;
        font-size: 16px;
        font-weight: bold;
        text-align: center;
      }}
    </style>
  </head>
  <body>
    <p><h2><strong>Hola {seller_name}, con ID {seller}! Estas son las órdenes que debes entregar hoy en tu punto de acopio asignado:</strong></h2></p></p>
    
    <br>
    {df_seller_seleccionada_6.to_html(index=False)}
    <br>

    <strong><p>Datos sobre tu punto de acopio:</strong></p>
    <ol>
        <li>Nombre del punto: {punto_dropoff}</li>
        <li>Tienda: {tienda}</li>
        <li>-	Dirección: {direccion}</li>
        <li>Horario de entrega: {horario}</li>
        <li>Frecuencia: {frecuencia}</li>
    </ol>
    
    <p>🚚 Revisa nuestro tutorial de despacho: <a href="https://youtu.be/lpk1_y9uYDg">Tutorial de despacho</a>.</p>
    
    
    <p><strong>¡Importante!</strong> </p>
        <li>Recuerda que el punto de acopio debe firmar el cargo de entrega que le proporciones.</li>
        <li>Las ordenes con 3 días de retraso de no ser entregadas el día de hoy,serán canceladas.</li>
        <li>Este correo es un refuerzo. Siempre debes entregar los pedidos gestionados a tiempo en su respectiva fecha límite de despacho.</li>
        
    <p><p>🌟 Si necesitas soporte personalizado a través de WhatsApp, contáctanos en el {phone_asignado} o <a href="{link_phone}">ingresando al siguiente link</a>.</p>
    <p>¡Contamos con tu apoyo para mejorar la experiencia de nuestros clientes!</p>
        
  </body>
</html>
  """

      # Adjuntar el cuerpo del correo en formato HTML
    msg.attach(MIMEText(mensaje_html, 'html'))

    # Enviar el correo electrónico
    server.sendmail(correo_usuario, msg['To'], msg.as_string())

# Cerrar la conexión con el servidor SMTP
server.quit()

(221, b'2.0.0 Service closing transmission channel')

# BLOQUE PARA ENVIAR MENSAJE DE WHATS APP A SELLERS AAA

In [79]:
mensaje_whatsapp_1 = """
*Hola {seller_Name}, Hoy estás en la ruta de recolección*

Detectamos que tienes órdenes poseen retraso en la entrega, es importante que las entregues hoy al operador logístico.

Para mayor detalle te hemos enviado un mail al correo *{correo}* con asunto "F.COM | 🚨 HOY ESTAS EN RUTA🚨"


🚚 Aquí podrás revisar el tutorial de gestión de órdenes: : https://youtu.be/lpk1_y9uYDg

Queremos reforzar contigo el proceso para gestionarlas:

📦 Para que podamos recolectar tu orden deberás:

1-	Podrás revisar tus órdenes en falabella seller center desde aquí: https://sellercenter.falabella.com/order
2-	Recuerda hacer clic en “Listo para despachar” como máximo un día antes de la fecha límite de despacho (antes de las 10p.m.)


🌟Si necesitas soporte personalizado a través de WhatsApp, contáctanos en el *{Phone}* o haz clic aquí {link_phone}

¡Gracias!

"""



for seller in info_backlog_5['seller_ID'].unique():
    # Filtrar el DataFrame para la tienda actual
    df_seller_4 = info_backlog_5[info_backlog_5['seller_ID'] == seller]
    
    
    Numero = df_seller_4['Numero'].iloc[0]
    
    
    whatsapp_message = mensaje_whatsapp_1.format(
                    seller_Name=df_seller_4["seller_Name"].iloc[0],
                    correo=df_seller_4["Correo"].iloc[0],
                    Phone =df_seller_4["loyaltyPhone"].iloc[0],
                    link_phone = df_seller_4['EnlaceAAA'].iloc[0]
            )
    #Enviar el mensaje de WhatsApp
    pywhatkit.sendwhatmsg_instantly(Numero, whatsapp_message)
    time.sleep(12)     


KeyboardInterrupt: 